In [3]:
import requests as req , re , pandas as pd
import networkx as nx 
from matplotlib import pyplot as plt
from pyvis.network import Network

In [4]:
def connect(source):
    url = 'https://t.me/s/' + source
    request = req.get(url)
    html = request.text
    print('connected to' , source)
    return html

In [5]:
def targets(html , source):
    regex = re.compile(r'href=.https://t.me/\w+')
    data = []
    for target in regex.findall(html):
        target = target.rsplit('/' , 1)[1]
        if target.lower() != source.lower():
            data.append([source, target])
    edge_df = pd.DataFrame(data , columns=['source', 'target'])
    print('got targets from' , source)
    return edge_df

In [6]:
def subs (html , source):
    regex = re.compile(r'class=.counter_value.>[^<]+')
    try:
        string = regex.findall(html)[0]
        number = convert_str_to_number(string.rsplit('>' , 1)[1])
        print('got ' , number ,  'subs from ' , source)
        return number
    except:
        print('cant get subs for' , source)
    return

note: connect , targets and subs functions can be edited per social network

In [7]:
def convert_str_to_number(x):
    total_stars = 0
    num_map = {'K':1000, 'M':1000000, 'B':1000000000}
    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
    return int(total_stars)

In [8]:
def finals(source):
    html = connect(source)
    size = subs(html , source)
    edge_df = targets(html , source)
    edge_df['source_node_size'] = size
    edge_df['edge_size'] = edge_df.groupby(['target'])['source'].transform('count')
    edge_df = edge_df.drop_duplicates(subset=['target'])
    print('passed finals for' , source)
    return edge_df

In [9]:
def loop(df , i , start_channel_name, iter_number):
    iter = iter_number
    if i < iter:
      final_df  = pd.DataFrame()
      targets = df['target']
      for target in targets:
          edge_df = finals(target)
          final_df = final_df.append(edge_df)
          print('current length = ' , len(final_df))
      if i+1==iter:
        final_df.to_csv('{} iteration for {}.csv'.format(str(i),start_channel_name))
      i += 1
      print('iteration' , i)
      loop(final_df , i , start_channel_name, iter)
      return
    else: return "finished"

In [ ]:
channel_name = input("Enter the channel name: ")
iterations_number = int(input("Enter the number of iterations: "))
loop(finals(channel_name) , 0 , channel_name, iterations_number)

In [11]:
got_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='whitw')

In [12]:
edge_df = pd.read_csv('{} iteration for {}.csv'.format(str(iterations_number-1), channel_name))

edge_data = zip(edge_df['source'], edge_df['target'], edge_df['edge_size'])

In [13]:
for row in edge_data:
    src = row[0]
    trgt = row[1]
    w = row[2]
    node_link = "<a href=\ 'http://t.me/s/" + src + "'>" + src + "</a>"
    got_net.add_node(src, src, title=node_link)
    got_net.add_node(trgt, trgt, title=trgt)
    got_net.add_edge(src, trgt, value=w)

neighbor_map = got_net.get_adj_list()
# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

In [14]:
got_net.show_buttons()
got_net.show('{} iteration for {}.html'.format(str(iterations_number-1),channel_name))